In [ ]:
import sys
import os


sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

RESULTS_PATH = 'results.csv'
OPTUNA_DATABASE = 'study_results.db'
BASE_CONFIGURATION_NAME = 'configuration'


In [ ]:
from automl.loggers.result_logger import ResultLogger
import optuna
import optuna.visualization as vis
from automl.utils.optuna_utils import load_study_from_database
import matplotlib.pyplot as plt


# Load the experiment

In [ ]:
#base_experiment_path = "C:\\Experiments\\rl-zoo-CartPole-dqn-2\\HPOptimizationExperiments\\3\\experiments"
#experiment_relative_path = 'original'

base_experiment_path = "C:\\ricardo-goncalo-thesis-project\\project\\examples\\hp_optimization\\data\\hp_lodaded_exps"
experiment_relative_path = "exp_115"

#base_experiment_path = "C:\\Experiments\\rl-zoo-CartPole-ppo-multi_thread\\HpExperiments\\1"
#experiment_relative_path = "experiment"

experiment_path = f'{base_experiment_path}\\{experiment_relative_path}'

In [ ]:
if not os.path.exists(experiment_path):
    raise Exception("DOES NOT EXIST")

In [ ]:
from automl.core.advanced_input_management import gen_component_from
from automl.hp_opt.hp_optimization_pipeline import HyperparameterOptimizationPipeline


hyperparameter_optimization_pipeline : HyperparameterOptimizationPipeline = gen_component_from(experiment_path)
hyperparameter_optimization_pipeline.change_logger_level("NONE") # we don't want any logging done while we're looking into it

# Evaluation of HyperparameterOptimizationPipeline

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_hp_opt_results_logger

hyperparameter_optimization_results = hyperparameter_optimization_pipeline.get_decoupled_results_logger()

print(f"Hyperparameter_optimization_results in path: {hyperparameter_optimization_results.get_artifact_directory()}")

In [ ]:
experiments_in_results = set(hyperparameter_optimization_results.get_dataframe()["experiment"].values)
component_indexes_in_results = set(hyperparameter_optimization_results.get_dataframe()["component_index"].values)

print(f"Experiments in results:\n{experiments_in_results}")
print(f"Component indexes per experiment:\n{component_indexes_in_results}")

In [ ]:
colors_available_for_component_indexes = ["red", "blue", "green"]
colors_for_component_indexes = {}

color_iter = iter(colors_available_for_component_indexes)
for component_index in component_indexes_in_results:
    colors_for_component_indexes[component_index] = next(color_iter)

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_hp_opt_optuna_study


optuna_study = hyperparameter_optimization_pipeline.get_study()
trials_in_optuna = optuna_study.get_trials()


## Hyperparameter Study

In [ ]:
try:
    print(f"optuna_study done with with best value {optuna_study.best_value} in trial {optuna_study.best_trial.number} with best parameters:\n{optuna_study.best_params}")

except:
    print("No best trial yet")

In [ ]:
parameters_to_plot = []
# parameters_to_plot = ["hidden_size", "hidden_layers"]

if len(parameters_to_plot) > 1:

    fig = vis.plot_contour(optuna_study, params=parameters_to_plot)
    fig.show()

# Global evaluation of configurations

In [ ]:
AGGREGATE_NUMBER = 10 #the number of neighbor points to sum to plot the needed graphs

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import study_of_configuration      
from automl.hp_opt.hp_eval_results.hp_eval_results import study_of_components_for_configuration      


In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_results_of_configurations_components                          

results_of_configurations : dict[str, ResultLogger] = get_results_of_configurations_components(experiment_path)

In [ ]:
print(f"Configurations:  {results_of_configurations.keys()}")

# Global view of performance

In [ ]:
hyperparameter_optimization_pipeline.proccess_input_if_not_proccesd()
config_dict_to_optimize = hyperparameter_optimization_pipeline.config_dict

In [ ]:
max_steps_to_do = config_dict_to_optimize["input"]["rl_trainer"][1]["limit_total_steps"]
steps_done_in_step = max_steps_to_do / hyperparameter_optimization_pipeline.n_steps

In [ ]:
from automl.rl.evaluators.rl_learning_evaluator import RLLearningEvaluatorSlope


evaluator_to_compare_to = RLLearningEvaluatorSlope()

In [ ]:
results = []

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_results_of_configuration_in_path
from automl.hp_opt.hp_opt_strategies.hp_optimization_loader_detached import COMPONENT_INDEX_TO_USE_OPTUNA_KEY, CONFIGURATION_PATH_OPTUNA_KEY


for trial in trials_in_optuna:
    
    trial : optuna.trial.FrozenTrial = trial
    best_index = int(trial.user_attrs.get(COMPONENT_INDEX_TO_USE_OPTUNA_KEY))
    
    if best_index is not None:
        results_logger_of_trial = get_results_of_configuration_in_path(
            os.path.join(trial.user_attrs.get(CONFIGURATION_PATH_OPTUNA_KEY), str(best_index)),
            "RLTrainerComponent"
        )

        print("-" * 10 + f"Trial: {trial.number}" + "-" * 10)
        
        for step, value in trial.intermediate_values.items():
            evaluator_to_compare_to.pass_input({
                "init" : (step * steps_done_in_step, "step"),
                "final" : (steps_done_in_step + step * steps_done_in_step, "step")
            })

            new_value = evaluator_to_compare_to.evaluate(results_logger_of_trial)

            results.append((trial.number, step, value, new_value))

            print(f"    Step: {step} had original value {value}, {new_value} with new evaluator")

        print()        

    else:
        raise NotImplementedError()

In [ ]:
print("Trials, step, value, new_value, ordered by value")

results_sorted = sorted(results, key=lambda x: x[2])

for trial_number, step, value, new_value in results_sorted:
    print(
        f"Trial {trial_number}, Step {step}, "
        f"value={value}, new_value={new_value}"
    )

In [ ]:
print("Trials, step, value, new_value, ordered by new_value")

results_sorted = sorted(results, key=lambda x: x[3])

for trial_number, step, value, new_value in results_sorted:
    print(
        f"Trial {trial_number}, Step {step}, "
        f"value={value}, new_value={new_value}"
    )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Extract values
values = np.array([r[2] for r in results], dtype=float)
new_values = np.array([r[3] for r in results], dtype=float)

def min_max_normalize(x):
    min_x = np.min(x)
    max_x = np.max(x)
    if max_x - min_x == 0:
        return np.zeros_like(x)
    return (x - min_x) / (max_x - min_x)

# Normalize both
normalized_values = min_max_normalize(values)
normalized_new_values = min_max_normalize(new_values)

normalized_results = list(
    zip(
        [r[0] for r in results],  # trial number
        [r[1] for r in results],  # step
        normalized_values,
        normalized_new_values,
    )
)

# Plot
x_axis = np.arange(len(results))

plt.figure()
plt.scatter(x_axis, normalized_values)
plt.scatter(x_axis, normalized_new_values)

plt.xlabel("Result Index")
plt.ylabel("Normalized Value")
plt.title("Normalized Original vs New Evaluator Values")
plt.legend(["Original (normalized)", "New (normalized)"])
plt.show()